# Task 3.1 ==> PYSPARK

In [1]:
#Import and initiate findspark Then import pyspark

import findspark

findspark.init("/usr/local/spark")

import pyspark

Starting spark session

In [2]:
from pyspark.sql import SparkSession
appName = "PySpark Hive Example"
master = "local"
# Create Spark session with Hive supported.
spark = SparkSession.builder \
.appName(appName) \
.master(master) \
.enableHiveSupport() \
.getOrCreate()

# 1 Develop a PySpark application to load data Spark DataFrames and save it into Hive tables on a Hadoop cluster in an optimized format. 

Loadind data from hdfs

In [3]:
CarSales=spark.read.csv('Car_sales_details_p3.csv', inferSchema=True,header=True)
 

In [4]:
region =spark.read.csv('State_region_mapping_p3.csv', inferSchema=True,header=True)

In [5]:
salesStatus=spark.read.csv('sales_Status_p3.csv', inferSchema=True,header=True)

In [6]:
CarSales.show()

+--------+--------------------+----+-------------+---------+--------------------+-------------+------+-----------+------------+------------+----------+-------+----------+-----+
|Sales_ID|                name|year|selling_price|km_driven|   State or Province|         City|  fuel|seller_type|transmission|       owner|   mileage| engine| max_power|seats|
+--------+--------------------+----+-------------+---------+--------------------+-------------+------+-----------+------------+------------+----------+-------+----------+-----+
|       1|Maruti Swift Dzir...|2014|       450000|   145500|District of Columbia|   Washington|Diesel| Individual|      Manual| First Owner| 23.4 kmpl|1248 CC|    74 bhp|    5|
|       2|Skoda Rapid 1.5 T...|2014|       370000|   120000|            New York|New York City|Diesel| Individual|      Manual|Second Owner|21.14 kmpl|1498 CC|103.52 bhp|    5|
|       3|Honda City 2017-2...|2006|       158000|   140000|            Illinois|      Chicago|Petrol| Individual| 

In [7]:
salesStatus.show()

+--------+----+
|Sales_ID|sold|
+--------+----+
|       1|   Y|
|       2|   Y|
|       3|   Y|
|       4|   Y|
|       5|   Y|
|       6|   Y|
|       7|   Y|
|       8|   Y|
|       9|   Y|
|      10|   Y|
|      11|   Y|
|      12|   Y|
|      13|   Y|
|      15|   Y|
|      16|   Y|
|      17|   Y|
|      18|   Y|
|      19|   Y|
|      20|   Y|
|      21|   Y|
+--------+----+
only showing top 20 rows



In [8]:
region.show()

+-------+--------------------+
| Region|   State or Province|
+-------+--------------------+
|Central|            Illinois|
|Central|             Indiana|
|Central|                Iowa|
|Central|              Kansas|
|Central|            Michigan|
|Central|           Minnesota|
|Central|            Missouri|
|Central|            Nebraska|
|Central|        North Dakota|
|Central|            Oklahoma|
|Central|        South Dakota|
|Central|               Texas|
|Central|           Wisconsin|
|   East|         Connecticut|
|   East|            Delaware|
|   East|District of Columbia|
|   East|               Maine|
|   East|            Maryland|
|   East|       Massachusetts|
|   East|       New Hampshire|
+-------+--------------------+
only showing top 20 rows



In [9]:
region.printSchema()

root
 |-- Region: string (nullable = true)
 |-- State or Province: string (nullable = true)



# DataTransformation

In [10]:
CarSales=CarSales.withColumnRenamed("State or Province","State_or_Province")
region=region.withColumnRenamed("State or Province","State_or_Province")
CarSales.printSchema()
#Renaming the column to make it a single word string

root
 |-- Sales_ID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- selling_price: integer (nullable = true)
 |-- km_driven: integer (nullable = true)
 |-- State_or_Province: string (nullable = true)
 |-- City: string (nullable = true)
 |-- fuel: string (nullable = true)
 |-- seller_type: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- owner: string (nullable = true)
 |-- mileage: string (nullable = true)
 |-- engine: string (nullable = true)
 |-- max_power: string (nullable = true)
 |-- seats: integer (nullable = true)



# joining the tables

In [11]:
#Joining the tables and displaying them
New_cars=CarSales.join(region,['State_or_Province'])
New_cars=CarSales.join(salesStatus,['Sales_ID'])
New_cars.show()

+--------+--------------------+----+-------------+---------+--------------------+-------------+------+-----------+------------+------------+----------+-------+----------+-----+----+
|Sales_ID|                name|year|selling_price|km_driven|   State_or_Province|         City|  fuel|seller_type|transmission|       owner|   mileage| engine| max_power|seats|sold|
+--------+--------------------+----+-------------+---------+--------------------+-------------+------+-----------+------------+------------+----------+-------+----------+-----+----+
|       1|Maruti Swift Dzir...|2014|       450000|   145500|District of Columbia|   Washington|Diesel| Individual|      Manual| First Owner| 23.4 kmpl|1248 CC|    74 bhp|    5|   Y|
|       2|Skoda Rapid 1.5 T...|2014|       370000|   120000|            New York|New York City|Diesel| Individual|      Manual|Second Owner|21.14 kmpl|1498 CC|103.52 bhp|    5|   Y|
|       3|Honda City 2017-2...|2006|       158000|   140000|            Illinois|      Chi

In [12]:
CarSales.show(5)

+--------+--------------------+----+-------------+---------+--------------------+-------------+------+-----------+------------+------------+----------+-------+----------+-----+
|Sales_ID|                name|year|selling_price|km_driven|   State_or_Province|         City|  fuel|seller_type|transmission|       owner|   mileage| engine| max_power|seats|
+--------+--------------------+----+-------------+---------+--------------------+-------------+------+-----------+------------+------------+----------+-------+----------+-----+
|       1|Maruti Swift Dzir...|2014|       450000|   145500|District of Columbia|   Washington|Diesel| Individual|      Manual| First Owner| 23.4 kmpl|1248 CC|    74 bhp|    5|
|       2|Skoda Rapid 1.5 T...|2014|       370000|   120000|            New York|New York City|Diesel| Individual|      Manual|Second Owner|21.14 kmpl|1498 CC|103.52 bhp|    5|
|       3|Honda City 2017-2...|2006|       158000|   140000|            Illinois|      Chicago|Petrol| Individual| 

# Save df to a new table in Hive 


In [13]:
 #saving the data frame into table to run the hive queries 
CarSales.write.mode("overwrite").saveAsTable('CarSales')
spark.sql("select * from CarSales").show() 

+--------+--------------------+----+-------------+---------+--------------------+-------------+------+-----------+------------+------------+----------+-------+----------+-----+
|Sales_ID|                name|year|selling_price|km_driven|   State_or_Province|         City|  fuel|seller_type|transmission|       owner|   mileage| engine| max_power|seats|
+--------+--------------------+----+-------------+---------+--------------------+-------------+------+-----------+------------+------------+----------+-------+----------+-----+
|       1|Maruti Swift Dzir...|2014|       450000|   145500|District of Columbia|   Washington|Diesel| Individual|      Manual| First Owner| 23.4 kmpl|1248 CC|    74 bhp|    5|
|       2|Skoda Rapid 1.5 T...|2014|       370000|   120000|            New York|New York City|Diesel| Individual|      Manual|Second Owner|21.14 kmpl|1498 CC|103.52 bhp|    5|
|       3|Honda City 2017-2...|2006|       158000|   140000|            Illinois|      Chicago|Petrol| Individual| 

In [14]:
#since we need fetch the cars sales details , we are trying to get sold cars (status 'Y')
salesStatus.write.mode("overwrite").saveAsTable("salesStatus")

spark.sql("select * from salesStatus").show() 

+--------+----+
|Sales_ID|sold|
+--------+----+
|       1|   Y|
|       2|   Y|
|       3|   Y|
|       4|   Y|
|       5|   Y|
|       6|   Y|
|       7|   Y|
|       8|   Y|
|       9|   Y|
|      10|   Y|
|      11|   Y|
|      12|   Y|
|      13|   Y|
|      15|   Y|
|      16|   Y|
|      17|   Y|
|      18|   Y|
|      19|   Y|
|      20|   Y|
|      21|   Y|
+--------+----+
only showing top 20 rows



In [15]:
region.write.mode("overwrite").saveAsTable("region")
spark.sql("select * from region").show() 

+-------+--------------------+
| Region|   State_or_Province|
+-------+--------------------+
|Central|            Illinois|
|Central|             Indiana|
|Central|                Iowa|
|Central|              Kansas|
|Central|            Michigan|
|Central|           Minnesota|
|Central|            Missouri|
|Central|            Nebraska|
|Central|        North Dakota|
|Central|            Oklahoma|
|Central|        South Dakota|
|Central|               Texas|
|Central|           Wisconsin|
|   East|         Connecticut|
|   East|            Delaware|
|   East|District of Columbia|
|   East|               Maine|
|   East|            Maryland|
|   East|       Massachusetts|
|   East|       New Hampshire|
+-------+--------------------+
only showing top 20 rows



In [16]:
 
New_cars.write.mode("overwrite").saveAsTable("New_cars")
 
spark.sql("select * from New_cars").show() 

+--------+--------------------+----+-------------+---------+--------------------+-------------+------+-----------+------------+------------+----------+-------+----------+-----+----+
|Sales_ID|                name|year|selling_price|km_driven|   State_or_Province|         City|  fuel|seller_type|transmission|       owner|   mileage| engine| max_power|seats|sold|
+--------+--------------------+----+-------------+---------+--------------------+-------------+------+-----------+------------+------------+----------+-------+----------+-----+----+
|       1|Maruti Swift Dzir...|2014|       450000|   145500|District of Columbia|   Washington|Diesel| Individual|      Manual| First Owner| 23.4 kmpl|1248 CC|    74 bhp|    5|   Y|
|       2|Skoda Rapid 1.5 T...|2014|       370000|   120000|            New York|New York City|Diesel| Individual|      Manual|Second Owner|21.14 kmpl|1498 CC|103.52 bhp|    5|   Y|
|       3|Honda City 2017-2...|2006|       158000|   140000|            Illinois|      Chi

In [22]:
New_cars.select('name', 'year', 'selling_price').show(10)


+--------------------+----+-------------+
|                name|year|selling_price|
+--------------------+----+-------------+
|Maruti Swift Dzir...|2014|       450000|
|Skoda Rapid 1.5 T...|2014|       370000|
|Honda City 2017-2...|2006|       158000|
|Hyundai i20 Sport...|2010|       225000|
|Maruti Swift VXI ...|2007|       130000|
|Hyundai Xcent 1.2...|2017|       440000|
|Maruti Wagon R LX...|2007|        96000|
|  Maruti 800 DX BSII|2001|        45000|
|    Toyota Etios VXD|2011|       350000|
|Ford Figo Diesel ...|2013|       200000|
+--------------------+----+-------------+
only showing top 10 rows



In [25]:
#filter data by null values
df_fuel=New_cars.filter(New_cars.fuel.isNotNull())
df_fuel.select("fuel").distinct().show()

+------+
|  fuel|
+------+
|Diesel|
|   CNG|
|   LPG|
|Petrol|
+------+



In [26]:
New_cars.dtypes

[('Sales_ID', 'int'),
 ('name', 'string'),
 ('year', 'int'),
 ('selling_price', 'int'),
 ('km_driven', 'int'),
 ('State_or_Province', 'string'),
 ('City', 'string'),
 ('fuel', 'string'),
 ('seller_type', 'string'),
 ('transmission', 'string'),
 ('owner', 'string'),
 ('mileage', 'string'),
 ('engine', 'string'),
 ('max_power', 'string'),
 ('seats', 'int'),
 ('sold', 'string')]

In [27]:
from pyspark.sql.types import *
Schema=StructType([
  StructField('Sales_ID',IntegerType(),nullable=True),
  StructField('name',StringType(),nullable=True),
  StructField('year',IntegerType(),nullable=True),
  StructField('selling_price',IntegerType(),nullable=True),
  StructField('km_driven',IntegerType(),nullable=True), 
  StructField('State or Province',StringType(),nullable=True),
  StructField('City',StringType(),nullable=True),
  StructField('fuel',StringType(),nullable=True),
  StructField('seller_type',StringType(),nullable=True),
  StructField('transmission',StringType(),nullable=True),
  StructField('owner',StringType(),nullable=True),
  StructField('mileage',DoubleType(),nullable=True),
  StructField('engine',IntegerType(),nullable=True),
  StructField('max_power',FloatType(),nullable=True),
  StructField('seats',IntegerType(),nullable=True),
  StructField('sold',StringType(),nullable=True),
  StructField('Region',StringType(),nullable=True)
  ])
df_Cars = spark.read.option("header",True).schema(Schema).csv('New_cars.csv')

checking for null values

In [28]:
df_unique=df_Cars.select(df_Cars.name.isNull()).show()


+--------------+
|(name IS NULL)|
+--------------+
|         false|
|         false|
|         false|
|         false|
|         false|
|         false|
|         false|
|         false|
|         false|
|         false|
|         false|
|         false|
|         false|
|         false|
|         false|
|         false|
|         false|
|         false|
|         false|
|         false|
+--------------+
only showing top 20 rows



In [29]:
df_Cars.show()

+--------+--------------------+----+-------------+---------+--------------------+----------+------+-----------+------------+------------+-------+------+---------+-----+----+------+
|Sales_ID|                name|year|selling_price|km_driven|   State or Province|      City|  fuel|seller_type|transmission|       owner|mileage|engine|max_power|seats|sold|Region|
+--------+--------------------+----+-------------+---------+--------------------+----------+------+-----------+------------+------------+-------+------+---------+-----+----+------+
|       1|Maruti Swift Dzir...|2014|       450000|   145500|District of Columbia|Washington|Diesel| Individual|      Manual| First Owner|   23.0|  1248|     74.0|    5|   Y|  East|
|      38|Hyundai Verna VTV...|2019|      1149000|     5000|District of Columbia|Washington|Petrol| Individual|      Manual| First Owner|   17.0|  1591|    121.3|    5|   Y|  East|
|      66|Ford Figo 1.2P Ti...|2017|       600000|    25000|District of Columbia|Washington|Pet

In [30]:
df_Cars.count()

7906

In [31]:
df_Cars.printSchema()

root
 |-- Sales_ID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- selling_price: integer (nullable = true)
 |-- km_driven: integer (nullable = true)
 |-- State or Province: string (nullable = true)
 |-- City: string (nullable = true)
 |-- fuel: string (nullable = true)
 |-- seller_type: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- owner: string (nullable = true)
 |-- mileage: double (nullable = true)
 |-- engine: integer (nullable = true)
 |-- max_power: float (nullable = true)
 |-- seats: integer (nullable = true)
 |-- sold: string (nullable = true)
 |-- Region: string (nullable = true)



# 3 Perform profiling of the data through PySpark and ensure that it is migrated correctly whereever the source is an RDBMS

Filling nan values with zeros

In [32]:
#Filling nan values with zeros
df_Cars.fillna(value=0)

DataFrame[Sales_ID: int, name: string, year: int, selling_price: int, km_driven: int, State or Province: string, City: string, fuel: string, seller_type: string, transmission: string, owner: string, mileage: double, engine: int, max_power: float, seats: int, sold: string, Region: string]

# unique values in different columns

In [33]:
#printing the unique values of all the columns in the dataset
df_Cars.select("name").distinct().collect()

[Row(name='Toyota Etios Liva GD'),
 Row(name='Datsun GO Plus A'),
 Row(name='Mahindra KUV 100 G80 K2'),
 Row(name='Mahindra Bolero 2011-2019 ZLX'),
 Row(name='Tata Indica V2 Turbomax DLS BS IV'),
 Row(name='Datsun GO Anniversary Edition'),
 Row(name='Tata Indica V2 DLE BSIII'),
 Row(name='Hyundai i20 1.4 Asta'),
 Row(name='Mahindra TUV 300 T4'),
 Row(name='Honda City 1.5 S MT'),
 Row(name='Maruti 800 DX BSII'),
 Row(name='Mahindra Bolero SLX 2WD BSIII'),
 Row(name='Maruti Ciaz 1.4 Alpha'),
 Row(name='Maruti Zen Classic'),
 Row(name='Mahindra Xylo D2 BS IV'),
 Row(name='Maruti Swift Lxi BSIII'),
 Row(name='Toyota Innova 2.5 VX (Diesel) 8 Seater BS IV'),
 Row(name='Mahindra Scorpio Intelli Hybrid S10'),
 Row(name='Maruti Zen LXI'),
 Row(name='Maruti Swift Dzire Vdi BSIV'),
 Row(name='Hyundai Verna 1.6 CRDI'),
 Row(name='Honda Brio V MT'),
 Row(name='Maruti Baleno Zeta'),
 Row(name='Hyundai Xcent 1.2 Kappa SX Option'),
 Row(name='Audi Q3 35 TDI Quattro Technology'),
 Row(name='Chevrolet S

In [34]:
df_Cars.select("Region").distinct().collect()

[Row(Region='South'),
 Row(Region='Central'),
 Row(Region='East'),
 Row(Region='West')]

In [35]:
df_Cars.select("city").distinct().collect()

[Row(city='Tyler'),
 Row(city='Hanover'),
 Row(city='Worcester'),
 Row(city='Agawam'),
 Row(city='Prattville'),
 Row(city='Springfield'),
 Row(city='Edmonds'),
 Row(city='Harrisburg'),
 Row(city='Tempe'),
 Row(city='Charleston'),
 Row(city='Bowling Green'),
 Row(city='Auburn'),
 Row(city='Harrison'),
 Row(city='Batavia'),
 Row(city='Wilkinsburg'),
 Row(city='Thornton'),
 Row(city='North Las Vegas'),
 Row(city='Plainfield'),
 Row(city='Lake Elsinore'),
 Row(city='Georgetown'),
 Row(city='Wilson'),
 Row(city='West Mifflin'),
 Row(city='Bethlehem'),
 Row(city='Camp Springs'),
 Row(city='Randallstown'),
 Row(city='Phoenix'),
 Row(city='Cranford'),
 Row(city='Cherry Hill'),
 Row(city='Merrimack'),
 Row(city='Levittown'),
 Row(city='Waukegan'),
 Row(city='Hollywood'),
 Row(city='Baldwin'),
 Row(city='Council Bluffs'),
 Row(city='Brunswick'),
 Row(city='Pleasantville'),
 Row(city='Pittsburg'),
 Row(city='Mount Prospect'),
 Row(city='Woodland'),
 Row(city='Menlo Park'),
 Row(city='Belchertown'

In [36]:
df_Cars.select("state or Province").distinct().collect()

[Row(state or Province='Utah'),
 Row(state or Province='Minnesota'),
 Row(state or Province='Ohio'),
 Row(state or Province='Oregon'),
 Row(state or Province='Arkansas'),
 Row(state or Province='Texas'),
 Row(state or Province='North Dakota'),
 Row(state or Province='Pennsylvania'),
 Row(state or Province='Connecticut'),
 Row(state or Province='Vermont'),
 Row(state or Province='Nebraska'),
 Row(state or Province='Nevada'),
 Row(state or Province='Washington'),
 Row(state or Province='Illinois'),
 Row(state or Province='Oklahoma'),
 Row(state or Province='District of Columbia'),
 Row(state or Province='Delaware'),
 Row(state or Province='New Mexico'),
 Row(state or Province='West Virginia'),
 Row(state or Province='Missouri'),
 Row(state or Province='Rhode Island'),
 Row(state or Province='Georgia'),
 Row(state or Province='Montana'),
 Row(state or Province='Michigan'),
 Row(state or Province='Virginia'),
 Row(state or Province='North Carolina'),
 Row(state or Province='Wyoming'),
 Row

# count of Cars that are sold in different cities

In [37]:
from pyspark.sql.functions import desc

# Display number of values in city column
df_Cars.groupBy('city').count().orderBy(desc("count")).show(df_Cars.count(), True)

+-------------------+-----+
|               city|count|
+-------------------+-----+
|      New York City|  196|
|        Los Angeles|  191|
|            Seattle|   92|
|            Chicago|   88|
|             Boston|   75|
|         Washington|   68|
|       Philadelphia|   57|
|              Miami|   45|
|          Charlotte|   45|
|            Detroit|   42|
|            Atlanta|   39|
|            Houston|   38|
|             Dallas|   36|
|          San Diego|   34|
|        Springfield|   26|
|           Columbus|   25|
|       Mount Vernon|   21|
|      San Francisco|   21|
|            Durango|   20|
|         Burlington|   20|
|           Richmond|   20|
|   Twentynine Palms|   20|
|            Sanford|   20|
|           Danville|   19|
|          Lancaster|   19|
|             Auburn|   18|
|         Prior Lake|   18|
|          Rochester|   17|
|        Gainesville|   17|
|               Troy|   17|
|       Indianapolis|   17|
|             Bangor|   17|
|             Newark

In [38]:
df_Cars.printSchema()

root
 |-- Sales_ID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- selling_price: integer (nullable = true)
 |-- km_driven: integer (nullable = true)
 |-- State or Province: string (nullable = true)
 |-- City: string (nullable = true)
 |-- fuel: string (nullable = true)
 |-- seller_type: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- owner: string (nullable = true)
 |-- mileage: double (nullable = true)
 |-- engine: integer (nullable = true)
 |-- max_power: float (nullable = true)
 |-- seats: integer (nullable = true)
 |-- sold: string (nullable = true)
 |-- Region: string (nullable = true)



In [39]:
from pyspark.sql import Row
from pyspark.sql.functions import min, max

In [40]:
df_Cars.select(min("year"),max("year")).show(df_Cars.count(), False)

+---------+---------+
|min(year)|max(year)|
+---------+---------+
|1994     |2020     |
+---------+---------+



In [41]:
df_Cars.select(min("selling_price"),max("selling_price")).show(df_Cars.count(), False)

+------------------+------------------+
|min(selling_price)|max(selling_price)|
+------------------+------------------+
|29999             |10000000          |
+------------------+------------------+



In [42]:
df_Cars.select(min("max_power"),max("max_power")).show(df_Cars.count(), False)

+--------------+--------------+
|min(max_power)|max(max_power)|
+--------------+--------------+
|32.8          |400.0         |
+--------------+--------------+



In [43]:
df_Cars.select(min("year"),max("year")).show(df_Cars.count(), False)

+---------+---------+
|min(year)|max(year)|
+---------+---------+
|1994     |2020     |
+---------+---------+



# mean and standard deviation for selling_price

In [44]:

from pyspark.sql.functions import mean as _mean, stddev as _stddev, col

df_Cars1 = df_Cars.select(
    _mean(col('selling_price')).alias('mean'),
    _stddev(col('selling_price')).alias('std')
).collect()

mean = df_Cars1[0]['mean']
std = df_Cars1[0]['std']




In [45]:
df_Cars1

[Row(mean=649813.720844928, std=813582.7483541325)]

In [46]:
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col

df_Cars2 = df_Cars.select(
    _mean(col('mileage')).alias('mean'),
    _stddev(col('mileage')).alias('std')
).collect()

mean = df_Cars2[0]['mean']
std = df_Cars2[0]['std']


In [47]:
df_Cars2

[Row(mean=19.18050594485198, std=4.0702331098045645)]

In [50]:
import pyspark.sql.functions as f

df_Cars.groupby('name').agg(f.count('name').alias('count')).show()

+--------------------+-----+
|                name|count|
+--------------------+-----+
|Toyota Etios Liva GD|    8|
|    Datsun GO Plus A|    2|
|Mahindra KUV 100 ...|    3|
|Mahindra Bolero 2...|   19|
|Tata Indica V2 Tu...|    2|
|Datsun GO Anniver...|    1|
|Tata Indica V2 DL...|    4|
|Hyundai i20 1.4 Asta|    2|
| Mahindra TUV 300 T4|    3|
| Honda City 1.5 S MT|    7|
|  Maruti 800 DX BSII|    1|
|Mahindra Bolero S...|    3|
|Maruti Ciaz 1.4 A...|    5|
|  Maruti Zen Classic|    1|
|Mahindra Xylo D2 ...|    3|
|Maruti Swift Lxi ...|    1|
|Toyota Innova 2.5...|    1|
|Mahindra Scorpio ...|    1|
|      Maruti Zen LXI|   11|
|Maruti Swift Dzir...|    7|
+--------------------+-----+
only showing top 20 rows



# correlation

In [51]:
#displaying the correlation between the different columns of the dataset 
from pyspark.ml.stat import Correlation
import pyspark.sql.functions as f
from pyspark.sql.functions import corr
df_Cars.agg(corr("mileage", "selling_price").alias('correlation')).collect()

[Row(correlation=-0.11809890195246613)]

In [52]:
df_Cars.agg(corr("max_power","selling_price").alias('correlation')).collect()

[Row(correlation=0.7496737776232365)]

In [53]:
df_Cars.agg(corr("seats","selling_price").alias('correlation')).collect()

[Row(correlation=0.04161669383026376)]

In [54]:
df_Cars.agg(corr("km_driven","selling_price").alias('correlation')).collect()

[Row(correlation=-0.22215847533483699)]

In [55]:
df_Cars.agg(corr("year","selling_price").alias('correlation')).collect()

[Row(correlation=0.41230155817117037)]

In [56]:
df_Cars.agg(corr("engine","selling_price").alias('correlation')).collect()

[Row(correlation=0.4556818000356127)]

# statistical summary for numerical columns

In [57]:
 #displaying the statisticak summary for everty column in the dataset
df_Cars.select('selling_price').describe().show()

+-------+-----------------+
|summary|    selling_price|
+-------+-----------------+
|  count|             7906|
|   mean| 649813.720844928|
| stddev|813582.7483541325|
|    min|            29999|
|    max|         10000000|
+-------+-----------------+



In [58]:
df_Cars.select('km_driven').describe().show()

+-------+-----------------+
|summary|        km_driven|
+-------+-----------------+
|  count|             7906|
|   mean|69188.65975208703|
| stddev|56792.29634331763|
|    min|                1|
|    max|          2360457|
+-------+-----------------+



In [59]:
df_Cars.select('max_power').describe().show()

+-------+-----------------+
|summary|        max_power|
+-------+-----------------+
|  count|             7906|
|   mean|91.58737369540249|
| stddev|35.74721595179561|
|    min|             32.8|
|    max|            400.0|
+-------+-----------------+



In [60]:
df_Cars.select('seats').describe().show()

+-------+------------------+
|summary|             seats|
+-------+------------------+
|  count|              7906|
|   mean|5.4163926132051605|
| stddev|0.9592082121984603|
|    min|                 2|
|    max|                14|
+-------+------------------+



In [61]:
df_Cars.select('mileage').describe().show()

+-------+------------------+
|summary|           mileage|
+-------+------------------+
|  count|              7906|
|   mean| 19.18050594485198|
| stddev|4.0702331098045645|
|    min|               0.0|
|    max|              42.0|
+-------+------------------+



In [67]:
df_US=[col for col,type in df_Cars.dtypes if type in ['int','float','double']]
cor_mat=[]
for col1 in df_US:
    temp=[]
    for col2 in df_US:
        temp.append(df_Cars.corr(col1,col2))
    cor_mat.append(temp)


In [68]:
for col in ['']+df_US: print("%10s"%col, end=' ')
print()
i=0
for col in df_US:
    print("%10s"%col,end=' ')
    for corr in cor_mat[i]:
        print("%10.2f"%corr,end=' ')
    print()
    i+= 1

             Sales_ID       year selling_price  km_driven    mileage     engine  max_power      seats 
  Sales_ID       1.00      -0.00      -0.01       0.01       0.03      -0.01      -0.00      -0.01 
      year      -0.00       1.00       0.41      -0.43       0.34       0.02       0.23      -0.01 
selling_price      -0.01       0.41       1.00      -0.22      -0.12       0.46       0.75       0.04 
 km_driven       0.01      -0.43      -0.22       1.00      -0.18       0.21      -0.04       0.23 
   mileage       0.03       0.34      -0.12      -0.18       1.00      -0.57      -0.36      -0.45 
    engine      -0.01       0.02       0.46       0.21      -0.57       1.00       0.70       0.61 
 max_power      -0.00       0.23       0.75      -0.04      -0.36       0.70       1.00       0.19 
     seats      -0.01      -0.01       0.04       0.23      -0.45       0.61       0.19       1.00 


In [70]:
df_US

['Sales_ID',
 'year',
 'selling_price',
 'km_driven',
 'mileage',
 'engine',
 'max_power',
 'seats']

In [71]:
from pyspark.sql.functions import countDistinct
df2=df_Cars.select(countDistinct( "Region"),countDistinct("name"),countDistinct("year"),countDistinct("selling_price"))
df2.show()

+----------------------+--------------------+--------------------+-----------------------------+
|count(DISTINCT Region)|count(DISTINCT name)|count(DISTINCT year)|count(DISTINCT selling_price)|
+----------------------+--------------------+--------------------+-----------------------------+
|                     4|                1982|                  27|                          670|
+----------------------+--------------------+--------------------+-----------------------------+



# 4 Write PySpark routines to cleanse the data, prepare the data to handle missing values, and the data transformations identified in task 1.1 again making sure that the data is written into Hive tables in an efficient format

In [72]:
from pyspark.sql.functions import split
df_Cars=df_Cars.withColumn('brand',split(df_Cars['name'],'\s')[0])

In [73]:
df_Cars.show()

+--------+--------------------+----+-------------+---------+--------------------+----------+------+-----------+------------+------------+-------+------+---------+-----+----+------+----------+
|Sales_ID|                name|year|selling_price|km_driven|   State or Province|      City|  fuel|seller_type|transmission|       owner|mileage|engine|max_power|seats|sold|Region|     brand|
+--------+--------------------+----+-------------+---------+--------------------+----------+------+-----------+------------+------------+-------+------+---------+-----+----+------+----------+
|       1|Maruti Swift Dzir...|2014|       450000|   145500|District of Columbia|Washington|Diesel| Individual|      Manual| First Owner|   23.0|  1248|     74.0|    5|   Y|  East|    Maruti|
|      38|Hyundai Verna VTV...|2019|      1149000|     5000|District of Columbia|Washington|Petrol| Individual|      Manual| First Owner|   17.0|  1591|    121.3|    5|   Y|  East|   Hyundai|
|      66|Ford Figo 1.2P Ti...|2017|    

In [74]:
from pyspark.sql.functions import split 
df_Cars= df_Cars.withColumnRenamed('State or Province' , 'state')
df_Cars.show()

+--------+--------------------+----+-------------+---------+--------------------+----------+------+-----------+------------+------------+-------+------+---------+-----+----+------+----------+
|Sales_ID|                name|year|selling_price|km_driven|               state|      City|  fuel|seller_type|transmission|       owner|mileage|engine|max_power|seats|sold|Region|     brand|
+--------+--------------------+----+-------------+---------+--------------------+----------+------+-----------+------------+------------+-------+------+---------+-----+----+------+----------+
|       1|Maruti Swift Dzir...|2014|       450000|   145500|District of Columbia|Washington|Diesel| Individual|      Manual| First Owner|   23.0|  1248|     74.0|    5|   Y|  East|    Maruti|
|      38|Hyundai Verna VTV...|2019|      1149000|     5000|District of Columbia|Washington|Petrol| Individual|      Manual| First Owner|   17.0|  1591|    121.3|    5|   Y|  East|   Hyundai|
|      66|Ford Figo 1.2P Ti...|2017|    

In [76]:
spark.sql("SELECT brand,COUNT(brand) FROM cars_Sales GROUP BY brand order by count(brand) desc").show(5)

+--------+------------+
|   brand|count(brand)|
+--------+------------+
|  Maruti|        2367|
| Hyundai|        1360|
|Mahindra|         758|
|    Tata|         719|
|   Honda|         466|
+--------+------------+
only showing top 5 rows



In [77]:
 #displaying the region wise count of sales data
spark.sql("SELECT Region,COUNT(*) FROM cars_Sales GROUP BY Region order by count(*) desc").show(5)

+-------+--------+
| Region|count(1)|
+-------+--------+
|Central|    2376|
|   West|    1976|
|   East|    1939|
|  South|    1615|
+-------+--------+



In [78]:
 #displaying the city wise count of sales data
spark.sql("SELECT city,COUNT(*) FROM cars_Sales GROUP BY city order by count(*) desc").show(5)

+-------------+--------+
|         city|count(1)|
+-------------+--------+
|New York City|     196|
|  Los Angeles|     191|
|      Seattle|      92|
|      Chicago|      88|
|       Boston|      75|
+-------------+--------+
only showing top 5 rows



In [79]:
 #displaying the state wise count of sales data
spark.sql("SELECT state,COUNT(*) FROM cars_Sales GROUP BY state order by count(*) desc").show(5)

+----------+--------+
|     state|count(1)|
+----------+--------+
|California|     901|
|     Texas|     499|
|  New York|     491|
|  Illinois|     489|
|   Florida|     425|
+----------+--------+
only showing top 5 rows



## 5) If the predictive model identified in task 2.4 available in Spark MLlib then develop a PySpark application to implement and evaluate the ML model identified with appropriate metrics

In [82]:
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
for col,type in df_Cars.dtypes:
    if type=='string':
        indexer = StringIndexer(inputCol=col, outputCol=col+"Index")
        df_Cars = indexer.fit(df_Cars).transform(df_Cars)
        df_Cars=df_Cars.drop(col)
df_Cars.show()

+--------+----+-------------+---------+-------+------+---------+-----+---------+----------+---------+---------+----------------+-----------------+----------+---------+-----------+----------+
|Sales_ID|year|selling_price|km_driven|mileage|engine|max_power|seats|nameIndex|stateIndex|CityIndex|fuelIndex|seller_typeIndex|transmissionIndex|ownerIndex|soldIndex|RegionIndex|brandIndex|
+--------+----+-------------+---------+-------+------+---------+-----+---------+----------+---------+---------+----------------+-----------------+----------+---------+-----------+----------+
|       1|2014|       450000|   145500|   23.0|  1248|     74.0|    5|      0.0|      33.0|      5.0|      0.0|             0.0|              0.0|       0.0|      1.0|        2.0|       0.0|
|      38|2019|      1149000|     5000|   17.0|  1591|    121.3|    5|    346.0|      33.0|      5.0|      1.0|             0.0|              0.0|       0.0|      1.0|        2.0|       1.0|
|      66|2017|       600000|    25000|   18.

In [81]:
df_Cars.printSchema()

root
 |-- Sales_ID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- selling_price: integer (nullable = true)
 |-- km_driven: integer (nullable = true)
 |-- state: string (nullable = true)
 |-- City: string (nullable = true)
 |-- fuel: string (nullable = true)
 |-- seller_type: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- owner: string (nullable = true)
 |-- mileage: double (nullable = true)
 |-- engine: integer (nullable = true)
 |-- max_power: float (nullable = true)
 |-- seats: integer (nullable = true)
 |-- sold: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- brand: string (nullable = true)



In [84]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


assembler = VectorAssembler(
    inputCols=["Sales_ID","nameIndex","year","km_driven","stateIndex","CityIndex","fuelIndex","seller_typeIndex","transmissionIndex","ownerIndex","mileage","engine","max_power","seats","soldIndex","RegionIndex","brandIndex"],
    outputCol="features")

# Split the data into training and test sets (30% held out for testing)
(training, test) = df_Cars.randomSplit([0.7, 0.3])

lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8,featuresCol='features',labelCol="selling_price")

# Fit the model
lrModel = lr.fit(training)

# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [0.24824352544351283,-210.29710188050367,35025.03728962152,-0.9210947248044399,-185.5615124077575,32.54320731563402,8415.294913878748,50121.75683161901,463835.6401824925,-728.0645779140756,13090.661885260024,131.7852088976598,12103.15058421388,-37747.36517322989,17816.342076025252,7994.961425441501,25223.70067391257]
Intercept: -71270116.59206624
numIterations: 11
objectiveHistory: [0.5000000000000001, 0.4368799591686584, 0.2408567124233807, 0.18671776959325592, 0.1697498228796912, 0.16282159086732131, 0.159090039428299, 0.15675562201466697, 0.15453230529241627, 0.15434182360240375, 0.15416579556891882]
+-------------------+
|          residuals|
+-------------------+
| -475825.1457211077|
| 369307.31745284796|
|  990316.3407754451|
| -96341.93357011676|
|-481838.11608687043|
| -56643.53535871208|
|  625016.8992832005|
| 62604.437858879566|
| 39957.627614200115|
| -5756.297112166882|
| 30941.587985157967|
|  335607.1151865274|
|  9944.753476083279|
|  89798.77241577208|
|

In [93]:
pred=lrModel.evaluate(test)
print(pred.meanSquaredError)

192581375938.08713
